# Analysis of NSW Food Authority's Name & Shame Register

The NSW Food Authority publishes lists of businesses that have breached or are alleged to have breached NSW food safety laws. Publishing the lists gives consumers more information to make decisions about where they eat or buy food. Individuals and businesses may receive either a penalty notice for their alleged offence or be prosecuted before a court.

In [1]:
# Libraries
import sys
sys.path.append('../utils')  # For notebooks
import utils 
import pandas as pd
import numpy as np
#import boto3
import os
import io
from dotenv import load_dotenv #for loading env variables
from github import Github #for pushing data to Github

## 1. Get Existing Data from Github

The Food Authority's Name & Shame website only displays the last 12 months of data. But since I started this repository (in June, 2024) I simply append any new data to the bottom of a dataset stored on Github. So step 1 of the overall process is to get this data

#### Get access keys and read from Github

In [2]:
# Load the environment variables from .env
load_dotenv()

# GitHub Authentication (Replace placeholders with your information)
access_token = os.environ.get("GITHUB_PERSONAL_ACCESS_TOKEN")
g = Github(access_token)

# Repository and File Information
repo_owner = "liampearson" 
repo_name = "nsw-food-authority-name-and-shame"
file_path = "data/dataset.csv"

# Get Repository
repo = g.get_user(repo_owner).get_repo(repo_name)

# Get File Contents
try:
    file_content = repo.get_contents(file_path)
    
    decoded_content = file_content.decoded_content.decode() # Decode if necessary    

    prev_df = pd.read_csv(io.StringIO(decoded_content))

    prev_df['notice_number'] = prev_df['notice_number'].astype(str) #convert to string for comparison
    print("   Dataset has been downloaded. Shape: {}\n".format(prev_df.shape))

except Exception as e:
    print(f"Error downloading file: {e}")

prev_df.head()

   Dataset has been downloaded. Shape: (681, 23)



,notice_number,trade_name,suburb,council,date,party_served,party_served_trade_name,address,city,postcode,...,offence_circumstances,penalty_amount,party_served_surname_company,penalty_date_served,penalty_issued_by,published_date,updated_date,party_served_given_name,scrape_timestamp_utc,date_removed_from_website
0,3120322920,SEHAJ INDIAN FOOD AND SWEETS,QUAKERS HILL,Blacktown,2023-04-25,RPST GROUP PTY LTD,SEHAJ INDIAN FOOD AND SWEETS,216 FARNHAM ROAD,QUAKERS HILL,2763,...,Fail to store food in such a way that it is pr...,880.0,RPST GROUP PTY LTD,2023-05-07,NaN,2023-07-18,2023-07-18,NaN,2024-06-22 00:12:46,2024-06-25
1,3120322930,SEHAJ INDIAN FOOD AND SWEETS,QUAKERS HILL,Blacktown,2023-04-25,RPST GROUP PTY LTD,SEHAJ INDIAN FOOD AND SWEETS,216 FARNHAM ROAD,QUAKERS HILL,2763,...,Fail to maintain the food premises to the requ...,880.0,RPST GROUP PTY LTD,2023-05-07,NaN,2023-07-18,2023-07-18,NaN,2024-06-22 00:12:46,2024-06-25
2,7682202963,VIET PHAT,BANKSTOWN,Canterbury-Bankstown,2023-05-03,VIET PHAT EXPRESS PTY LTD,VIET PHAT,45 BANKSTOWN CITY PLAZA,BANKSTOWN,2200,...,Fail to take all practicable measures to ensur...,880.0,VIET PHAT EXPRESS PTY LTD,2023-05-11,NaN,2023-07-18,2023-07-18,NaN,2024-06-22 00:12:46,2024-06-25
3,7682202954,A AND A FRUIT AND VEGETABLE MARKET,BANKSTOWN,Canterbury-Bankstown,2023-05-03,TRAN,A AND A FRUIT AND VEGETABLE MARKET,55 BANKSTOWN CITY PLAZA,BANKSTOWN,2200,...,Fail to take all practicable measures to ensur...,440.0,TRAN,2023-05-11,Canterbury Bankstown Council,2023-07-18,2023-07-18,NaN,2024-06-22 00:12:46,2024-06-25
4,7682202752,JASMINE LEBANESE RESTAURANT,LAKEMBA,Canterbury-Bankstown,2023-03-06,S AND F KITCHENER PTY LTD,JASMINE LEBANESE RESTAURANT,30B HALDON STREET,LAKEMBA,2195,...,Fail to store potentially hazardous food under...,880.0,S AND F KITCHENER PTY LTD,2023-04-13,NaN,2023-07-25,2023-07-25,NaN,2024-06-22 00:12:46,2024-06-25


## 2. Get all notices that are currently on the Food Authority Website

The function `scrape_tables` takes a url (which we've defined as the food authority Name and Shame Register) and iterates over child-page of the website. 

The result is `notice_df`; a dataframe of all the notices across all pages of the parent url. 

In [3]:
#the parent page we are going to scrape
url = "https://www.foodauthority.nsw.gov.au/offences/penalty-notices"

print("iterate over the pages of url:\n  {}\n".format(url))
#scrape each of the pages and get the table of notices
notice_df = utils.scrape_tables(url)

iterate over the pages of url:
  https://www.foodauthority.nsw.gov.au/offences/penalty-notices

   processing page 1 (index: 0)...
   processing page 2 (index: 1)...
   processing page 3 (index: 2)...
   processing page 4 (index: 3)...
   processing page 5 (index: 4)...
   processing page 6 (index: 5)...
   processing page 7 (index: 6)...
   processing page 8 (index: 7)...
   processing page 9 (index: 8)...
   processing page 10 (index: 9)...
   processing page 11 (index: 10)...
   processing page 12 (index: 11)...
   processing page 13 (index: 12)...
   processing page 14 (index: 13)...
   processing page 15 (index: 14)...
   no tables on page 15 (index: 14)



## 3. Compare the website to dataset

We will now compare the notices found in Step 2, to the notices we already have in step 1

an `old_notice_number` is one which, as of last scrape, had not yet been removed from the website. We know what hadn't been removed by filtering on only those which have a null `date_removed_from_website`. 

A `current_notice_number` is any notice live on the website now. 

In [4]:
old_notice_numbers = prev_df[prev_df['date_removed_from_website'].isnull()]['notice_number'].tolist()

current_notice_numbers = notice_df['notice_number'].tolist()

#get the difference of the above to determine new and removed notices.
removed_notice_numbers = set(old_notice_numbers) - set(current_notice_numbers)
new_notice_numbers = set(current_notice_numbers) - set(old_notice_numbers)

print("{} notice_numbers removed".format(len(removed_notice_numbers)))
print("{} notice_numbers added".format(len(new_notice_numbers)))

0 notice_numbers removed
0 notice_numbers added


We now do a number of checks and run certain code based on these checks. 

E.g:
 * if **a notice was removed**
     * update the `date_removed_from_website` field
 * if **a notice was added**
     * open up a particular page to get the finer details
     * then append to the dataset

In [5]:
#check if notice numbers were removed
if len(removed_notice_numbers)==0:
    print("   0 notice_numbers removed")
    
else:
    print("   {} notice_numbers removed".format(len(removed_notice_numbers)))
    prev_df = utils.handle_removed_notices(prev_df, removed_notice_numbers)

#check if any new notice numbers
if len(new_notice_numbers)==0:
    print("   0 new notice_numbers added")
    result = prev_df #since no new entries, the result is just the old dataframe. 
    
else:
    print("   {} new notice_numbers found".format(len(new_notice_numbers)))
    print(new_notice_numbers)
    
    #we'll only work with these
    notice_df = notice_df[notice_df['notice_number'].isin(new_notice_numbers)]
    
    #check they're unique
    #check only unique numbers
    if not len(notice_df['notice_number'].unique()) == len(notice_df):
        raise ValueError("Not all policy numbers are unique")
        
    else: #Get details per notice_number
        print("4. Get penalty info...")
        #empty list to collect each row as a dictionary
        penalties = []

        for notice_number in new_notice_numbers:
            print("   processing: {}".format(notice_number))

            # scrape the website
            record = utils.get_penalty_notice(notice_number)    
            penalties.append(record)
            
        print("Complete\n")

        penalties_df = pd.DataFrame(penalties)
        
        utils.cleanup_dataframe(penalties_df)
        notice_df = utils.join_dataframes(penalties_df, notice_df)
        notice_df = utils.add_timestamp(notice_df)

   0 notice_numbers removed
   0 new notice_numbers added


## 4. Finalise the dataset and push back to Github

In [6]:
# boolean flag which triggers an upload or not
need_to_upload=False

if len(new_notice_numbers)>0:
    need_to_upload=True
    # add the new notices to the previous dataframe
    result = pd.concat([prev_df, notice_df], ignore_index=True)

if len(removed_notice_numbers)>0:
    need_to_upload=True

#if its ben identifed that the data needs to be uploaded
if need_to_upload:
    result.sort_values(by=['published_date', 'council', 'suburb', 'trade_name'], inplace=True, ascending=[False, True, True,True])
    
    #overwrite dataset
    print("5. Begin Upload to Github...")    
    repo = g.get_user(repo_owner).get_repo(repo_name)
    
    # Update the main dataset
    file_content = result.to_csv(index=False)  # Convert to CSV

    try:
        # Check if file exists
        contents = repo.get_contents(file_path)
        repo.update_file(contents.path, "Updated dataset", file_content, contents.sha)
        print(f"Updated existing file: {file_path}")

    except:  # File doesn't exist
        repo.create_file(file_path, "Added dataset", file_content)
        print(f"Created new file: {file_path}") 

#There were no changes to data so no need to upload.
else:
    print("no changes to the website so dataset will not be updated at this time.")

result.head()

no changes to the website so dataset will not be updated at this time.


,notice_number,trade_name,suburb,council,date,party_served,party_served_trade_name,address,city,postcode,...,offence_circumstances,penalty_amount,party_served_surname_company,penalty_date_served,penalty_issued_by,published_date,updated_date,party_served_given_name,scrape_timestamp_utc,date_removed_from_website
0,3120322920,SEHAJ INDIAN FOOD AND SWEETS,QUAKERS HILL,Blacktown,2023-04-25,RPST GROUP PTY LTD,SEHAJ INDIAN FOOD AND SWEETS,216 FARNHAM ROAD,QUAKERS HILL,2763,...,Fail to store food in such a way that it is pr...,880.0,RPST GROUP PTY LTD,2023-05-07,NaN,2023-07-18,2023-07-18,NaN,2024-06-22 00:12:46,2024-06-25
1,3120322930,SEHAJ INDIAN FOOD AND SWEETS,QUAKERS HILL,Blacktown,2023-04-25,RPST GROUP PTY LTD,SEHAJ INDIAN FOOD AND SWEETS,216 FARNHAM ROAD,QUAKERS HILL,2763,...,Fail to maintain the food premises to the requ...,880.0,RPST GROUP PTY LTD,2023-05-07,NaN,2023-07-18,2023-07-18,NaN,2024-06-22 00:12:46,2024-06-25
2,7682202963,VIET PHAT,BANKSTOWN,Canterbury-Bankstown,2023-05-03,VIET PHAT EXPRESS PTY LTD,VIET PHAT,45 BANKSTOWN CITY PLAZA,BANKSTOWN,2200,...,Fail to take all practicable measures to ensur...,880.0,VIET PHAT EXPRESS PTY LTD,2023-05-11,NaN,2023-07-18,2023-07-18,NaN,2024-06-22 00:12:46,2024-06-25
3,7682202954,A AND A FRUIT AND VEGETABLE MARKET,BANKSTOWN,Canterbury-Bankstown,2023-05-03,TRAN,A AND A FRUIT AND VEGETABLE MARKET,55 BANKSTOWN CITY PLAZA,BANKSTOWN,2200,...,Fail to take all practicable measures to ensur...,440.0,TRAN,2023-05-11,Canterbury Bankstown Council,2023-07-18,2023-07-18,NaN,2024-06-22 00:12:46,2024-06-25
4,7682202752,JASMINE LEBANESE RESTAURANT,LAKEMBA,Canterbury-Bankstown,2023-03-06,S AND F KITCHENER PTY LTD,JASMINE LEBANESE RESTAURANT,30B HALDON STREET,LAKEMBA,2195,...,Fail to store potentially hazardous food under...,880.0,S AND F KITCHENER PTY LTD,2023-04-13,NaN,2023-07-25,2023-07-25,NaN,2024-06-22 00:12:46,2024-06-25


## Create and push subset of latest notices to Github

In [11]:
latest_results = result[result['published_date']==result['published_date'].max()][['published_date', 'council','trade_name', 'address', 'suburb', 'offence_code',
       'offence_circumstances', 'penalty_amount','penalty_amount','trade_name']]

# Update the latest_result dataset
file_content = latest_results.to_csv(index=False)  # Convert to CSV
file_path = "data/last_weeks_notices.csv"

try:
    # Check if file exists
    contents = repo.get_contents(file_path)
    repo.update_file(contents.path, "Updated dataset", file_content, contents.sha)
    print(f"Updated existing file: {file_path}")

except:  # File doesn't exist
    repo.create_file(file_path, "Added dataset", file_content)
    print(f"Created new file: {file_path}") 

Updated existing file: data/last_weeks_notices.csv
